<a href="https://colab.research.google.com/github/isurunuwanthilaka/ml-house-price/blob/master/house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install kaggle
!mkdir /root/.kaggle

!echo '{"username":"isurunuwanthilaka","key":"db2148ff93a08f45957b3b1ef11040fe"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!mkdir /content
!kaggle config path -p /content
!kaggle competitions download -c house-prices-advanced-regression-techniques

mkdir: cannot create directory ‘/content’: File exists
usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 12.2MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 53.0MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 60.2MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 13.3MB/s]


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot
%matplotlib inline
from sklearn.model_selection  import  train_test_split
import xgboost


In [0]:
test = pd.read_csv('./test.csv')
train = pd.read_csv('./train.csv')

In [0]:
ytrain = train['SalePrice']
xtrain = train.drop('SalePrice',axis=1)